In [2]:
import os
import biosapi
import cobrakbase
from cobrakbase.core import KBaseFBAModel

cobrakbase 0.2.2


In [3]:
kbase = cobrakbase.KBaseAPI('UGOG6KLAWTCYI2ASYECYHNIIFTEXGA2J')
bios_live = biosapi.BIOS()
ws_fungi = 'jplfaria:narrative_1510597445008'
CACHE_BASE_FOLDER = '/Users/fliu/workspace/jupyter/python3/annotation-server/data/'

In [5]:
%run ../../scripts/bios_utils.py

In [6]:
models = [
    'iNL895',
    'iCT646',
    
    'iMM904',
    'iTO977',
    'iSS884',
    'iLC915',
    
    'iWV1213',
    'iAL1006',
    
    'iRL766',
    'iMA871',
    
    'iJDZ836',
    'iWV1314',
    'iOD907',
    'iJL1454',
    'iNX804',
    'yeast_6.06',
    'yeast_7.6',
]

In [7]:
def get_bios_tax_lineage(genome):
    bios_tax_lineage = list(map(lambda x : x.strip(), genome['taxonomy'].split('; ')))
    bios_tax_lineage.append(genome['scientific_name'])
    bios_tax_lineage.reverse()
    bios_tax_lineage.append('root')
    return bios_tax_lineage

def build_fungi_list(bios, kbase, fungi_ws):
    bios_model_list = bios.get_models()
    bios_model_dict = {}
    for o in bios_model_list:
        if 'entry' in o:
            bios_model_dict[o['entry']] = o
            
    bios_model_dict['iNX804'] = {
        'entry' : 'iNX804',
        'proxy' : False,
    }
    bios_model_dict['iWV1314'] = {
        'entry' : 'iWV1314',
        'proxy' : False,
    }
    bios_model_dict['iJL1454'] = {
        'entry' : 'iJL1454',
        'proxy' : False,
    }
    bios_model_dict['iWV1213'] = {
        'entry' : 'iWV1213',
        'proxy' : False,
    }
    
    bios_model_dict['yeast_6.06']['bios_tax_lineage'] = bios_model_dict['iMM904']['bios_tax_lineage']
    bios_model_dict['yeast_7.6']['bios_tax_lineage'] = bios_model_dict['iMM904']['bios_tax_lineage']
    bios_model_dict['yeast_7.6']['bios_refseq'] = bios_model_dict['iMM904']['bios_refseq']
    bios_model_dict['yeast_6.06']['bios_refseq'] = bios_model_dict['iMM904']['bios_refseq']
    
    found = set(bios_model_dict) & set(models)
    missing = set(models) - set(bios_model_dict)
    print(missing)
    print(found)
    
    #fungi_ws = 'jplfaria:narrative_1510597445008'
    kmodels = {}
    fbamodels = {}
    for model_id in models:
        kmodel = kbase.get_object(model_id + '_KBase2', fungi_ws)
        kmodels[model_id] = kmodel
        fbamodels[model_id] = KBaseFBAModel(kmodel)
    
    for model_id in found:
        data = bios_model_dict[model_id]
        id = model_id
        proxy = data['proxy']
        if not 'bios_tax_lineage' in data:
            genome_ref = fbamodels[model_id].data['genome_ref']
            ref = kbase.get_object_info_from_ref(genome_ref)
            if ref.id + '.json' in os.listdir(CACHE_BASE_FOLDER + '/cache/genomes'):
                genome = read_json(CACHE_BASE_FOLDER + '/cache/genomes/' + ref.id + '.json')
                data['bios_refseq'] = {
                    'entry' : genome['id']
                }
                data['bios_tax_lineage'] = get_bios_tax_lineage(genome)
                print(id, ref.id)
        bios_tax_lineage = '?' if not 'bios_tax_lineage' in data else data['bios_tax_lineage'][0]

        print(proxy, id, bios_tax_lineage)
        
    taxa = [
        'root',
        'cellular organisms',
     'Eukaryota',
     'Opisthokonta',
     'Fungi',
     'Fungi incertae sedis',
     'Mucoromycota',
     'Mucoromycotina',
     'Mucoromycetes',
     'Mucorales',
     'Mucorineae',
     'Mucoraceae',
     'Mucor',
     'Mucor circinelloides',
     'Mucor circinelloides f. lusitanicus',
     'Mucor circinelloides f. lusitanicus CBS 277.49'
           ]
    taxa.reverse()
    bios_model_dict['iWV1213']['bios_tax_lineage'] = taxa
    
    l = []
    for model_id in found:
        data = bios_model_dict[model_id]
        l.append(data)
        
    return l

fungi_model_list = build_fungi_list(bios_live, kbase, ws_fungi)
#write_json(fungi_model_list, CACHE_BASE_FOLDER + '/bios_cache_list_fungi.json', True)

set()
{'iOD907', 'iNX804', 'iWV1213', 'iSS884', 'iMA871', 'iNL895', 'iWV1314', 'iMM904', 'iLC915', 'iTO977', 'iJDZ836', 'iJL1454', 'iCT646', 'yeast_7.6', 'iRL766', 'iAL1006', 'yeast_6.06'}
iOD907 GCF_000002515.2
False iOD907 Kluyveromyces lactis NRRL Y-1140
iNX804 GCF_000002545.3
False iNX804 Candida glabrata CBS 138
iWV1213 Mucor_circinelloides_CBS277.49_v2.0
False iWV1213 unknown_taxon
iSS884 GCF_000209165.1
False iSS884 Scheffersomyces stipitis CBS 6054
iMA871 GCF_000002855.3
False iMA871 Aspergillus niger CBS 513.88
iNL895 GCF_000002525.2
False iNL895 Yarrowia lipolytica CLIB122
iWV1314 GCF_000184455.2
False iWV1314 Aspergillus oryzae RIB40
False iMM904 Saccharomyces cerevisiae S288C
iLC915 GCF_000027005.1
False iLC915 Komagataella phaffii GS115
False iTO977 Saccharomyces cerevisiae S288C
iJDZ836 GCF_000182925.2
False iJDZ836 Neurospora crassa OR74A
iJL1454 GCF_000149615.1
False iJL1454 Aspergillus terreus NIH2624
iCT646 GCF_000006335.2
False iCT646 Candida tropicalis MYA-3404
Fals

In [8]:
len(fungi_model_list)

17